In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [22]:
from os import listdir
import glob
csvs = glob.glob('./Player A-*.csv')
csvs

['./Player A-moves-status-Sun Oct 20 09:41:02 2019.csv',
 './Player A-moves-status-Sun Oct 20 09:39:42 2019.csv',
 './Player A-moves-status-Sun Oct 20 09:49:42 2019.csv',
 './Player A-moves-status-Sat Oct 19 20:36:40 2019.csv',
 './Player A-moves-status-Sun Oct 20 09:46:24 2019.csv',
 './Player A-moves-status-Sun Oct 20 09:37:02 2019.csv',
 './Player A-moves-status-Sat Oct 19 19:08:52 2019.csv',
 './Player A-moves-status-Sun Oct 20 09:48:28 2019.csv',
 './Player A-moves-status-Sat Oct 19 19:29:14 2019.csv',
 './Player A-moves-status-Sun Oct 20 09:45:31 2019.csv']

In [23]:
data = pd.DataFrame()
cols = [f'c{i}' for i in range(25)] + ['status', 'attack']
for f in csvs:
    df = pd.read_csv(f, index_col=None, header=None, names=cols)
    data = pd.concat([data, df])
data.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c17,c18,c19,c20,c21,c22,c23,c24,status,attack
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,N/A,A2
1,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,Miss,D3
2,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,Miss,B4
3,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,Miss,E1
4,0,0,0,0,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,Hit,D1


In [24]:
# drop status 'won' because game is over and we don't have to predict
df = data.dropna(how='any') 
print(f"dropped {len(data) - len(df)} rows")

dropped 10 rows


In [25]:
df.status.unique()
df.status = df.status.apply(str.strip).apply(str.lower).apply(lambda v: v.replace(' ', '_')).apply(lambda v: 'na' if v == 'n/a' else v)
df.attack = df.attack.apply(str.strip).apply(str.lower)
df.head()

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c17,c18,c19,c20,c21,c22,c23,c24,status,attack
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,na,a2
1,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,miss,d3
2,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,miss,b4
3,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,miss,e1
4,0,0,0,0,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,hit,d1


# First try with attack column as target with 25 classes

In [6]:
df = pd.get_dummies(df, columns=['status', 'attack'])
df.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,attack_d1,attack_d2,attack_d3,attack_d4,attack_d5,attack_e1,attack_e2,attack_e3,attack_e4,attack_e5
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,-1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [7]:
y = df[df.columns[df.columns.str.contains('attack')]]
X = df.drop(columns=y.columns)
y.head(), X.head()

(   attack_a1  attack_a2  attack_a3  attack_a4  attack_a5  attack_b1  \
 0          0          1          0          0          0          0   
 1          0          0          0          0          0          0   
 2          0          0          0          0          0          0   
 3          0          0          0          0          0          0   
 4          0          0          0          0          0          0   
 
    attack_b2  attack_b3  attack_b4  attack_b5  ...  attack_d1  attack_d2  \
 0          0          0          0          0  ...          0          0   
 1          0          0          0          0  ...          0          0   
 2          0          0          1          0  ...          0          0   
 3          0          0          0          0  ...          0          0   
 4          0          0          0          0  ...          1          0   
 
    attack_d3  attack_d4  attack_d5  attack_e1  attack_e2  attack_e3  \
 0          0          0      

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)
len(X_train), len(X_test)

(118, 30)

In [20]:
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [21]:
classifier.score(X_test, y_test)

0.0

# Second try with attack column split into attack_col and attack_row
### one model to predict col and another model to predict row

In [29]:
df.attack.value_counts()

e1    10
e2    10
a3    10
a2     8
c4     7
d4     7
a1     7
d2     6
e3     6
b2     6
e4     6
e5     6
c3     6
b3     6
d1     5
c5     5
b5     5
d5     5
b4     4
c1     4
c2     4
b1     4
d3     4
a4     4
a5     3
Name: attack, dtype: int64

In [15]:
# split attack into attack_col and attack_row to train two models one to detect col and another model to detect row
exp_cols = df.attack.apply(lambda v: v[0] + ' ' + v[1]).str.split(expand=True)
df['attack_col'], df['attack_row'] = exp_cols[0], exp_cols[1]
df.drop(columns=['attack'], inplace=True)
df.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c18,c19,c20,c21,c22,c23,c24,status,attack_col,attack_row
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,na,a,2
1,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,miss,d,3
2,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,miss,b,4
3,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,miss,e,1
4,0,0,0,0,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,hit,d,1


In [16]:
df['attack_row'].unique()
df = pd.get_dummies(df, columns=['status', 'attack_col', 'attack_row'])
df.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,attack_col_a,attack_col_b,attack_col_c,attack_col_d,attack_col_e,attack_row_1,attack_row_2,attack_row_3,attack_row_4,attack_row_5
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,0,0,0,0,0,-1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
4,0,0,0,0,1,-1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [17]:
y = df.loc[:, df.columns.str.contains('attack_col')]
X = df.drop(columns=df.columns[df.columns.str.contains('attack_')])
y.head(), X.head()

(   attack_col_a  attack_col_b  attack_col_c  attack_col_d  attack_col_e
 0             1             0             0             0             0
 1             0             0             0             1             0
 2             0             1             0             0             0
 3             0             0             0             0             1
 4             0             0             0             1             0,
    c0  c1  c2  c3  c4  c5  c6  c7  c8  c9  ...  c19  c20  c21  c22  c23  c24  \
 0   0   0   0   0   0   0   0   0   0   0  ...    0    0    0    0    0    0   
 1   0   0   0   0   0  -1   0   0   0   0  ...    0    0    0    0    0    0   
 2   0   0   0   0   0  -1   0   0   0   0  ...    0    0    0    0    0    0   
 3   0   0   0   0   0  -1   0   0   0   0  ...    0    0    0    0    0    0   
 4   0   0   0   0   1  -1   0   0   0   0  ...    0    0    0    0    0    0   
 
    status_hit  status_miss  status_na  status_ship_sunk  
 0           0

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
len(X_train), len(X_test)

(118, 30)

In [19]:
col_classifier = RandomForestClassifier(n_estimators=100)
col_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [20]:
col_classifier.score(X_test, y_test)

0.2

In [21]:
col_classifier.score(X_train, y_train)

0.940677966101695

# Todo
1. check if target classes are balanced
2. confusion matrix error analysis
3. learning curve
4. -1,0,+1  instead 0-1
5. training error
6. data leakage
7. split test data using entire new game